In [192]:
import numpy as np
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.spatial.distance import cosine

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
%run model_functions.ipynb

In [4]:
genre = 'academic'
all_words = pd.read_csv('data/train/{}/words.csv'.format(genre), na_filter=False)
display(len(all_words))
all_words.head()

48964

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,25070,HER,she,DPS,,,1267,a6u-fragment02,a6u,academic
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic


In [5]:
all_words['text_id'].nunique(), all_words['sentence_id'].nunique()

(10, 2023)

We have 10 texts and 2023 sentences. Let's try to find 2 unique topics per text!

Let's also only consider content words, so we don't consider words like "the", "and", "be".

In [69]:
content_words = build_content_words(all_words)
display(len(content_words))
content_words.head()

25360

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic
6,25076,KAHLO,kahlo,NP0,,,1267,a6u-fragment02,a6u,academic


In [84]:
def get_topic_assignments(components, vocab):
    n_top_words = 10
    topic_words = {}
    
    for topic, component in enumerate(components):
        # Argsort and reverse iterate to get vocab indices of top words for each topic. 
        top_word_ids = np.argsort(component)[::-1][:n_top_words]

        # Store the words most relevant to the topic
        topic_words[topic] = [vocab.iloc[i].name for i in top_word_ids]
        
    for topic, words in topic_words.items():
        print('Topic: %d' % topic)
        print('  %s' % ', '.join(words))

In [240]:
# Turn lemmas of document to a document word matrix.
def document_word_matrix(words, vocabulary):
    indices = []
    data = []
    indptr = [0]
    for i, word in words['lemma'].iteritems():
        # Used to generate a row of 0's and 1's for sparse matrix
        unique_unigram_indices = set()
        # ID for an unknown word is == len(vocabulary).
        if word not in vocabulary.index:
            unique_unigram_indices.add(len(vocabulary))
        else:
            unique_unigram_indices.add(vocabulary.index.get_loc(word))

        # Data = 1 at column index of sparse matrix = row index of word in vocabulary.
        indices.extend(unique_unigram_indices)  # Indices of the data values in this row.
        data.extend(np.ones(len(unique_unigram_indices), dtype=int))  # Data of all one's.
        indptr.append(indptr[-1]+len(unique_unigram_indices))

        if not i % 2000:
            print('-', end='', flush=True)

    return csr_matrix((data, indices, indptr), dtype=int, shape=(len(words), len(vocabulary)+1))

Assumption: lemmatized content words -> better topics.

In [241]:
len(content_words)

25360

In [242]:
sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')[['sentence_id', 'paragraph_id']]
dataset = (
    content_words
     .merge(sentences)
     .merge(pd.Series(build_paragraphs(content_words).index))
)
display(len(dataset))
dataset.head()  # The words that belong to paragraphs.

25029

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre,paragraph_id
0,25083,witnessed,witness,VVN,mrw,met,1269,a6u-fragment02,a6u,academic,412
1,25085,shift,shift,NN1-VVB,,,1269,a6u-fragment02,a6u,academic,412
2,25088,art,art,NN1,,,1269,a6u-fragment02,a6u,academic,412
3,25089,establishment,establishment,NN1,,,1269,a6u-fragment02,a6u,academic,412
4,25090,attitudes,attitude,NN2,mrw,met,1269,a6u-fragment02,a6u,academic,412


In [ ]:
def model_all_topics(dataset, n_topics=20, max_iter=2000, n_jobs=-1):
    vocab = dataset[['lemma']].drop_duplicates().set_index('lemma')

    doc_topic_dists = []

    X = document_word_matrix(dataset, vocab)
    print(X.shape, len(dataset), len(vocab))

    lda = LatentDirichletAllocation(
        n_components=n_topics, max_iter=max_iter, random_state=0,
        verbose=10, n_jobs=n_jobs
    )

#     doc_topic_dist = lda.fit_transform(X)
    lda.fit(X)
    
    return lda
    
lda_model = model_all_topics(dataset, 20, 2000, -1)

-------------(25029, 5399) 25029 5398


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 2000


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 2000


...

iteration: 668 of max_iter: 2000


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 669 of max_iter: 2000


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 670 of max_iter: 2000


In [254]:
for paragraph, group in dataset.groupby('paragraph_id'):
    sparse_matrix = document_word_matrix(group, vocab)
    paragraph_topic_dist = lda_model.transform(sparse_matrix)
    t_given_w = calc_t_given_w(lda_model.components_, paragraph_topic_dist)
    t_given_d = calc_t_given_d(paragraph_topic_dist)
    break

-

ValueError: The provided data has 5443 dimensions while the model was trained with feature size 82.

In [108]:
# def assign_topics(components):
#     return [np.argmax(w_given_t) for w_given_t in zip(*components)]

In [75]:
# n_topics
def calc_t_given_d(doc_topic_dist):
    total_d = sum((sum(dtd) for dtd in doc_topic_dist))
    t_given_d = [
        sum(tdd)/total_d for tdd in zip(*doc_topic_dist)
    ]
    return t_given_d

# n_topics X n_vocabulary
def calc_w_given_t(components):
    return components / components.sum(axis=1)[:, np.newaxis]

# n_vocabulary
def calc_w(components):
    return components.sum(axis=0)/sum(sum(arr) for arr in components)

# n_topics
def calc_t(doc_topic_dist):
    return doc_topic_dist.sum(axis=0)/sum(sum(arr) for arr in doc_topic_dist)

# n_vocabulary X n_topics
def calc_t_given_w(components, doc_topic_dist):
    w_given_t = calc_w_given_t(components)
    p_w = calc_w(components)
    p_t = calc_t(doc_topic_dist)

    t_given_w = list(zip(*((w_given_t / p_w) * p_t[:,None])))
    return t_given_w

In [196]:
t_given_w = calc_t_given_w(lda.components_, doc_topic_dist)
document_topic_dist = calc_t_given_d(doc_topic_dist)

for i, word_topic_dist in enumerate(t_given_w):
    print(cosine(word_topic_dist, t_given_d))

0.7703997444772843
0.4568608040847908
0.5682705456894483
0.5969801809208133
0.7711934985706879
0.704236825302738
0.5682705456894483
0.5895686683001795
0.7232457050239681
0.7167114568487325
0.7681132522683601
0.7456774659871248
0.7495257035318393
0.7696857140290414
0.7517989488731738
0.6704324346563864
0.766484442627741
0.7286653132303739
0.5762918645369302
0.7605372021436756
0.7712254759138633
0.7730476396576952
0.7062049911719795
0.5334102804001908
0.5809102136492412
0.741671347013992
0.7522123442010502
0.7144738667685069
0.7553145466381573
0.7670946197115509
0.7763645094767722
0.5946975227988478
0.7759360157506578
0.5508091498245198
0.7286653132303739
0.5724995577370641
0.5764503284940196
0.6812782808465253
0.728604422283142
0.7454044625402965
0.7713789953818697
0.5895686683001795
0.7656611613608237
0.7554345302930096
0.7558342611824493
0.7523138798182591
0.7710947114566467
0.6700988816019597
0.7233477457158844
0.7754218817567788
0.7565332631040369
0.7560405013068735
0.74525413199449

0.5633728824251352
0.7406449947540207
0.7252183453172644
0.5607457512481041
0.7145863751533267
0.5574576527108244
0.7648662980482828
0.7284180175288673
0.7246878876060138
0.7538947655442216
0.6858979724522634
0.7034433491942513
0.7211341850703037
0.5469610443469163
0.5879837346256183
0.7310594947698761
0.7660496222181304
0.7560405013068735
0.7505534786992389
0.767969876921964
0.5882619029585566
0.7333983348037758
0.5882619029585566
0.5800798443301842
0.7119492267515878
0.7062049911719795
0.6729029586238544
0.7131617993556615
0.7636742836978112
0.5882619029585566
0.7441858725525188
0.5807869814176605
0.7286653132303739
0.5800798443301842
0.7407058638337968
0.7618175627852596
0.5147729468054624
0.7481477725168575
0.5974175387717684
0.740442003372735
0.5800798443301842
0.7614340826009695
0.5807869814176605
0.5946975227988478
0.5080200959694623
0.7062542971837974
0.7079596280564683
0.7628977769916463
0.7333983348037758
0.5548234858544188
0.5594585794705098
0.7038373640024058
0.705901411949

0.5564876262077356
0.6858979724522634
0.6787438297553232
0.5762918645369302
0.7534389133212537
0.7494805276667833
0.7386551657975229
0.5724995577370641
0.6788859729410153
0.7131617993556615
0.5767147831050021
0.5969801809208133
0.7120538327636279
0.5762918645369302
0.5946975227988478
0.5882619029585566
0.6797382291954442
0.7110223510428871
0.5682705456894483
0.7144738667685069
0.5767147831050021
0.5900349620925973
0.5767147831050021
0.7038373640024058
0.5800798443301842
0.5682705456894483
0.7675223456497439
0.7290672934417475
0.704236825302738
0.7749502120566527
0.7719980583255007
0.7430157404751048
0.7465691301850299
0.7062544378094144
0.5682705456894483
0.7246878876060138
0.7335970454124859
0.7505832886082879
0.5574576527108244
0.6853752062365983
0.7386551657975229
0.7119075516607931
0.7233477457158844
0.7456774659871248
0.6753072897864598
0.5594347078110169
0.584292671736899
0.6699307741063061
0.7470290743632766
0.728604422283142
0.7739092563092709
0.6340444123683129
0.7404991585641

0.5807869814176605
0.6729029586238544
0.5574576527108244
0.5879837346256183
0.5809102136492412
0.6704324346563864
0.7206226055619134
0.7167114568487325
0.7729963883156495
0.6700988816019597
0.6797382291954442
0.5922332575295298
0.5682705456894483
0.6731679590622207
0.5969801809208133
0.6791940685008921
0.5574576527108244
0.5682705456894483
0.5767147831050022
0.5724995577370641
0.5900349620925973
0.7211341850703037
0.740442003372735
0.733597045412486
0.5724995577370641
0.5682705456894483
0.6791940685008921
0.5809102136492412
0.744247383282836
0.5895686683001795
0.5767147831050022
0.5767147831050022
0.6731679590622207
0.5895686683001795
0.6791940685008921
0.5682705456894483
0.5807869814176605
0.7169992318427127
0.5891955453574895
0.5764503284940196
0.5891955453574895
0.7393997531861984
0.5682705456894483
0.6829107859906598
0.7110223510428871
0.7079596280564683
0.7608848245236925
0.7602039255705783
0.5922332575295298
0.5093755206015309
0.7119492267515878
0.5882619029585566
0.5900349620925

0.5946975227988478
0.7316429453989772
0.5724995577370641
0.6788859729410153
0.7063021869104704
0.5974175387717684
0.5969801809208133
0.5891955453574895
0.5812505968077692
0.5807869814176605
0.5812505968077692
0.5922332575295298
0.5807869814176605
0.5682705456894483
0.5895686683001795
0.5891955453574895
0.5969801809208133
0.5809102136492412
0.5724995577370641
0.5574576527108244
0.5724995577370641
0.5812505968077692
0.5969801809208133
0.5891955453574895
0.5882619029585566
0.6797909539898066
0.5974175387717684
0.5762918645369302
0.5969801809208133
0.5895686683001795
0.6788859729410153
0.5809102136492412
0.5879837346256183
0.6853752062365983
0.6797382291954442
0.5762918645369302
0.5764503284940196
0.5767147831050022
0.6729029586238544
0.5891955453574895
0.5767147831050022
0.5922332575295298
0.5767147831050021
0.7169992318427127
0.5800798443301842
0.5974175387717684
0.5762918645369302
0.5895686683001795
0.5922332575295298
0.5682705456894483
0.5879837346256183
0.5724995577370641
0.5764503284